In [1]:
!nvidia-smi

Tue Jun 10 10:24:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Package download

!pip install sentencepiece -q
!pip install transformers -q
!pip install datasets -q
!pip install peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platfo

## Part1

In [3]:
# Nllb loading

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
# model_name = "facebook/nllb-200-3.3B" # Larger model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.src_lang = "zho_Hant"
tokenizer.tgt_lang = "tgl_Latn"
# zho_Hant for Chinese traditional
# eng_Latn for English
# tgl_Latn for Puyuma (Use existing language tag)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)6cea38b9e3d5efcdcb9c251d6b40538e1aab555a:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)b3c438311629547285129b0b81dadabd01bca665:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

...

2025-06-10 10:26:24.673960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749551185.086143      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749551185.226357      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)1ecdf1e485509035f6b51dfe84f1ada83eefcc42:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
# Load data into dataframes

import pandas as pd

lexicon = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/lexicon_no_en.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
lexicon.columns = ['pyu', 'zho']

lexicon_en = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/lexicon.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
lexicon_en.columns = ['pyu', 'eng', 'zho']

sentences = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/sentences_no_en.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
sentences.columns = ['pyu', 'zho']

sentences_en = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/sentences.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
sentences_en.columns = ['pyu', 'eng', 'zho']

#lexicon.sample(5)
#lexicon_en.sample(10)
#sentences.sample(5)
#sentences_en.sample(10)

In [5]:
# Testing the performances of original tokenization

import re

def word_tokenize(text):
    
    return re.findall('(\w+|[^\w\s])', text)

def df_tokenize(df):
    df['pyu_toks'] = df.pyu.apply(tokenizer.tokenize)
    df['zho_toks'] = df.zho.apply(tokenizer.tokenize)
    df['pyu_words'] = df.pyu.apply(word_tokenize)
    df['zho_words'] = df.zho.apply(word_tokenize)
    
    return df

def cal_tokperword(df):

    stats = df[['pyu_toks', 'zho_toks', 'pyu_words', 'zho_words']].map(len).describe()
    print(stats.pyu_toks['mean'] / stats.pyu_words['mean'])
    print(stats.zho_toks['mean'] / stats.zho_words['mean'])

    return stats

def check_unk(df, column):

    texts_with_unk = [
        text for text in df[column]
        if tokenizer.unk_token_id in tokenizer(text).input_ids
    ]
    print(len(texts_with_unk))

lexicon = df_tokenize(lexicon)
lexicon_en = df_tokenize(lexicon_en)
sentences = df_tokenize(sentences)
sentences_en = df_tokenize(sentences_en)

print("toks per word of lexicon:")
stats_lexicon = cal_tokperword(lexicon)
print("toks per word of lexicon_en:")
stats_lexicon = cal_tokperword(lexicon_en)
print("toks per word of sentences:")
stats_sentences = cal_tokperword(sentences)
print("toks per word of sentences_en:")
stats_sentences = cal_tokperword(sentences_en)

print("total unk in lexicon zho:")
check_unk(lexicon, "zho")
print("total unk in lexicon pyu:")
check_unk(lexicon, "pyu")
print("total unk in lexicon_en zho:")
check_unk(lexicon_en, "zho")
print("total unk in lexicon_en pyu:")
check_unk(lexicon_en, "pyu")
print("total unk in sentences zho:")
check_unk(sentences, "zho")
print("total unk in sentences pyu:")
check_unk(sentences, "pyu")
print("total unk in sentences_en zho:")
check_unk(sentences_en, "zho")
print("total unk in sentences pyu:")
check_unk(sentences_en, "pyu")

#show datas
#lexicon.sample(10)
#sentences.sample(10)
#stats_lexicon
#stats_sentences

toks per word of lexicon:
2.2840203274985886
2.3545659526493803
toks per word of lexicon_en:
1.738603297769156
2.2698515171078117
toks per word of sentences:
1.5806060606060606
3.925910765452312
toks per word of sentences_en:
1.5050038491147038
3.165337423312884
total unk in lexicon zho:
97
total unk in lexicon pyu:
0
total unk in lexicon_en zho:
203
total unk in lexicon_en pyu:
0
total unk in sentences zho:
223
total unk in sentences pyu:
36
total unk in sentences_en zho:
142
total unk in sentences pyu:
14


In [6]:
# Training tokenizer for missing tokens

import pandas as pd
from tqdm.auto import tqdm
import re
from collections import Counter
import sentencepiece as spm
from datasets import load_dataset

all_texts = lexicon['zho'].dropna().tolist() + sentences['zho'].dropna().tolist() + lexicon_en['zho'].dropna().tolist() + sentences_en['zho'].dropna().tolist() + lexicon['pyu'].dropna().tolist() + sentences['pyu'].dropna().tolist() + lexicon_en['pyu'].dropna().tolist() + sentences_en['pyu'].dropna().tolist()

all_texts_file = 'all_texts_plain.txt'
with open(all_texts_file, 'w', encoding='utf-8') as f:
    for text in all_texts:
        print(text, file=f)

required_chars = set()

for text in tqdm(all_texts):
    for char in text:
        tokens = tokenizer.tokenize(char)
        if tokens == ['▁', '<unk>']:
            required_chars.add(char)

required_chars_str = "".join(sorted(list(required_chars)))
print(f"需要強制包含的單字元: {required_chars_str}")

spm.SentencePieceTrainer.train(
    input=all_texts_file,
    model_prefix='spm_new',
    vocab_size=5800,
    character_coverage=1,
    num_threads=16,
    train_extremely_large_corpus=False,
    add_dummy_prefix=False,
    max_sentencepiece_length=128,
    max_sentence_length=4192 * 4,
    pad_id=0,
    eos_id=1,
    unk_id=2,
    bos_id=-1,
    required_chars=required_chars_str,
)

  0%|          | 0/8770 [00:00<?, ?it/s]

需要強制包含的單字元: ’“”仟傻儼兇兜冀凳刮剁剝劈勻厲叢叮吱吵吼咀咕咱哇哎唉唬唷啃啄啥喂喔嗚嗯嘻噁噎嚀嚏嚕嚥嚨嚼囉囑坍垮墮墾壑夾妝妳姑姨婿媳嫂嬸宋寞屁屎屹岔峨峭峽嶺嶼巍巒廚廨彎徊徘怔怡惱懶扁扛扯抖拂拇拌拚捻掀掐揉揍揹搓搔搗搥摺撐撥撩撬擠擲攀攜攤攪攬敞晃晉晾暈杓杵柑柚桐梳椒椽楊榔槌樑樸橘橡檜檳櫻殷氓氾汙洶涎涕淒湃湊湛溉溼滷漱漾漿潑澀澆澈澎濁濕濤瀉瀑灑灶炒烘烹煞煥熬燉燙燦燻爐犁猴琉瓢甕甩畝疊疤痠痰癢皂盈盪眶睏睜瞄瞌瞧瞰矚砌碩碴碾磚礱稻窩竄竊竿笆筌筍筷箏箕篩篳簍簷籃籬粥粿糠糬糯糰緻縷繡繩羌羸耆耍耙聆脖腋膛膿臀臼舀舅舔舖芋芙苣苧茅莓菇菸萵葵蓆蔔蔗蔥蕃蕉蕎蕗蕨薑薯藷蘿蚓蚤蚯蚱蛀蛙蜓蜢蜻蝦蝨蝴蝸螂螃螞蟑蟬蟹蟻蠅裙諧諱謠謾豎豹贛趴跛踢蹂蹦蹲躪辣迴逛遐邈邵鄒酗醃醬釀鈔鉤鋤鋸錘錶鍊鍬鎚鏟鏽鐮鐺鑰閒閩闆闡闢阱陡雉霜鞦韆颱颳飧餚餵饋駝驟髒鬍鬚鬧魄鳳鴨鴿鵝鹹鹽麴齒龜


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 


In [7]:
# Add trained tokens to tokenizer and model

from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
from transformers import NllbTokenizer

model_name = 'facebook/nllb-200-distilled-600M'
tokenizer_nllb = NllbTokenizer.from_pretrained(model_name)

sp_trained = spm.SentencePieceProcessor(model_file='spm_new.model')
added_spm = sp_pb2_model.ModelProto()
added_spm.ParseFromString(sp_trained.serialized_model_proto())
old_spm_nllb = sp_pb2_model.ModelProto()
old_spm_nllb.ParseFromString(tokenizer_nllb.sp_model.serialized_model_proto())

nllb_tokens_set = {p.piece for p in old_spm_nllb.pieces}
prev_min_score = old_spm_nllb.pieces[-1].score
for p in added_spm.pieces:
    piece = p.piece
    if p.type != 1:
        continue
    if piece not in nllb_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = p.score + prev_min_score
        old_spm_nllb.pieces.append(new_p)

NEW_SPM_NAME = 'spm_nllb_extended_268k.model'
with open(NEW_SPM_NAME, 'wb') as f:
    f.write(old_spm_nllb.SerializeToString())

tokenizer = NllbTokenizer.from_pretrained(model_name, vocab_file='spm_new.model')
print(len(tokenizer_nllb), len(tokenizer))
added_vocab = set(tokenizer.get_vocab()).difference(set(tokenizer_nllb.get_vocab()))
#print(added_vocab)(0)

256204 6004


## PART2

In [8]:
lexicon.sample(5)

,pyu,zho,pyu_toks,zho_toks,pyu_words,zho_words
1267,pinu'utilran,裝有,"[▁p, inu, ', uti, l, ran]","[▁, 裝, 有]","[pinu, ', utilran]",[裝有]
1002,ututratrawan,待人接物,"[▁ut, ut, ratra, wan]","[▁, 待, 人, 接, 物]",[ututratrawan],[待人接物]
923,kalre'ayan,方便,"[▁kal, re, ', ayan]","[▁, 方, 便]","[kalre, ', ayan]",[方便]
57,risem,鐵撬,"[▁ris, em]","[▁, 鐵, <unk>]",[risem],[鐵撬]
469,parepidapidan,多次,"[▁par, ep, id, ap, idan]","[▁多, 次]",[parepidapidan],[多次]


In [9]:
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
model.cuda();
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)

In [10]:
import random
LANGS = [('zho', 'zho_Hant'), ('pyu', 'tgl_Latn')]

dfs = [lexicon, sentences, lexicon_en, sentences_en]
df_train = pd.concat([df[['pyu', 'zho']] for df in dfs], ignore_index=True)

def get_batch_pairs(batch_size, data=df_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(item[l1])
        yy.append(item[l2])
    return xx, yy, long1, long2

#print(get_batch_pairs(1))

In [11]:
batch_size = 16  # 32 already doesn't fit well to 15GB of GPU memory
max_length = 128  # token sequences will be truncated
training_steps = 50000  # Usually, I set a large number of steps,
# and then just interrupt the training manually
losses = []  # with this list, I do very simple tracking of average loss
MODEL_SAVE_PATH = '/kaggle/working/nllb_extended'  # on my Google drive

In [12]:
import gc
import torch
import numpy as np
from tqdm.auto import tqdm, trange

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

model.train()
x, y, loss = None, None, None
cleanup()

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        # -100 is a magic value ignored in the loss function
        # because we don't want the model to learn to predict padding ids
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:  # usually, it is out-of-memory
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 1000 == 0:
        # each 1000 steps, I report average loss at these steps
        print(i, np.mean(losses[-1000:]))

    if i % 1000 == 0 and i > 0:
        model.save_pretrained(MODEL_SAVE_PATH)
        tokenizer.save_pretrained(MODEL_SAVE_PATH)

  0%|          | 0/2000 [00:00<?, ?it/s]

0 10.844038009643555
1000 6.1033971543312076


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


## PART3

In [14]:
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM
model_load_name = '/kaggle/working/nllb_extended'
model = AutoModelForSeq2SeqLM.from_pretrained(model_load_name).cuda()
tokenizer = NllbTokenizer.from_pretrained(model_load_name)

In [27]:
def translate(
    text, src_lang='zho_Hant', tgt_lang='tgl_Latn', 
    a=32, b=3, max_input_length=1024, num_beams=1, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True, 
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

t = '今天是星期一'
print(translate(t, 'zho_Hant', 'tgl_Latn'))

["'a'a'a'a'"]


In [23]:
sentences.sample(10)

,pyu,zho,pyu_toks,zho_toks,pyu_words,zho_words
264,nu uri paveray pakangaway za muketrep wariyan ...,在婚期前十天，男家邀集男性親屬一同搬送聘禮至女家。,"[▁nu, ▁uri, ▁pa, ver, ay, ▁pakanga, way, ▁za, ...","[▁在, 婚, 期, 前, 十, 天, ,, 男, 家, 邀, 集, 男性, 親, 屬, 一...","[nu, uri, paveray, pakangaway, za, muketrep, w...","[在婚期前十天, ，, 男家邀集男性親屬一同搬送聘禮至女家, 。]"
22,"na Pinuyumayan mu, 'azi kazuwan tu pinitrauwan...",卑南族人口雖不多，但卻很團結。卑南族有這麼優良的傳統文化，身為卑南族人，要好好珍惜，並擔任起...,"[▁na, ▁P, inu, yum, ayan, ▁mu, ,, ▁', azi, ▁ka...","[▁, 卑, 南, 族, 人口, 雖, 不, 多, ,, 但, 卻, 很, 團, 結, 。,...","[na, Pinuyumayan, mu, ,, ', azi, kazuwan, tu, ...","[卑南族人口雖不多, ，, 但卻很團結, 。, 卑南族有這麼優良的傳統文化, ，, 身為卑南..."
862,"2. puitrasaw lra na ayawan, arasaw lra na ayaw...",2.把頭目（領袖）抬上來了，把頭目（領袖）抬起來了，虛詞...…,"[▁2., ▁pu, itr, asaw, ▁l, ra, ▁na, ▁aya, wan, ...","[▁2., 把, 頭, 目, (, 領, 袖, ), 抬, 上, 來, 了, ,, 把, 頭...","[2, ., puitrasaw, lra, na, ayawan, ,, arasaw, ...","[2, ., 把頭目, （, 領袖, ）, 抬上來了, ，, 把頭目, （, 領袖, ）, ..."
426,liyusan pukalruwatr wari? kiyumalra ku ziya ka...,週五嗎？我得問問其他人。,"[▁liy, usan, ▁puk, al, ru, wat, r, ▁wari, ?, ▁...","[▁, 週, 五, 嗎, ?, 我, 得, 問, 問, 其他人, 。]","[liyusan, pukalruwatr, wari, ?, kiyumalra, ku,...","[週五嗎, ？, 我得問問其他人, 。]"
330,kani i Pinuyumayan na Kasavakan zi na Katratri...,其中卑南族的建和與知本部落因往南遷徙時與昔日的排灣族已有血緣之故，木雕雷同的風格別具特色。,"[▁kani, ▁i, ▁P, inu, yum, ayan, ▁na, ▁Kasa, v,...","[▁其中, 卑, 南, 族, 的, 建, 和, 與, 知, 本, 部落, 因, 往, 南, ...","[kani, i, Pinuyumayan, na, Kasavakan, zi, na, ...","[其中卑南族的建和與知本部落因往南遷徙時與昔日的排灣族已有血緣之故, ，, 木雕雷同的風格別..."
458,ulra lra mu alrak?,你們有孩子了嗎？,"[▁ul, ra, ▁l, ra, ▁mu, ▁al, rak, ?]","[▁你們, 有, 孩子, 了, 嗎, ?]","[ulra, lra, mu, alrak, ?]","[你們有孩子了嗎, ？]"
757,"kinalrakan murairaip mu'uma, menadanadam mele'...",從小結伴做農，學習編織及料理家務，也接受婦女道德傳統禮儀的訓練。,"[▁kin, al, rakan, ▁mura, ira, ip, ▁mu, ', uma,...","[▁從, 小, 結, 伴, 做, 農, ,, 學習, 編, 織, 及, 料理, 家, 務, ...","[kinalrakan, murairaip, mu, ', uma, ,, menadan...","[從小結伴做農, ，, 學習編織及料理家務, ，, 也接受婦女道德傳統禮儀的訓練, 。]"
870,tu nana'uwan kana melrivak 'alrayan zantu ipar...,保障原住民族之平等地位及自主發展，,"[▁tu, ▁nana, ', uwan, ▁kana, ▁mel, ri, vak, ▁'...","[▁, 保障, 原, 住, 民族, 之, 平等, 地位, 及, 自, 主, 發展, ,]","[tu, nana, ', uwan, kana, melrivak, ', alrayan...","[保障原住民族之平等地位及自主發展, ，]"
208,nu pararahan za ruma' pakangawayan za sa'ami p...,在蓋房子的前一年就要準備木柱及黃藤，尋找堅固實心不容易腐朽不會白蟻蛀掉的，三十、五十甚至一百...,"[▁nu, ▁par, arahan, ▁za, ▁ruma, ', ▁pakanga, w...","[▁在, 蓋, 房, 子, 的, 前, 一年, 就要, 準備, 木, 柱, 及, 黃, 藤,...","[nu, pararahan, za, ruma, ', pakangawayan, za,...","[在蓋房子的前一年就要準備木柱及黃藤, ，, 尋找堅固實心不容易腐朽不會白蟻蛀掉的, ，, ..."
297,na pa'ira'iraw mu iniyam na zekalr i Pinuyumay...,所謂pa’ira’iraw的古謠是卑南族部落在年祭時男子所吟唱的歌謠。,"[▁na, ▁pa, ', ira, ', ira, w, ▁mu, ▁ini, yam, ...","[▁所, 謂, pa, <unk>, ira, <unk>, ira, w, 的, 古, <...","[na, pa, ', ira, ', iraw, mu, iniyam, na, zeka...","[所謂pa, ’, ira, ’, iraw的古謠是卑南族部落在年祭時男子所吟唱的歌謠, 。]"
